**Mount Drive**

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import libraries and installing libraries**

In [0]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F

In [2]:
pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=ba9cc8f089fe83045af6150bf9212aec0ccb541ae84e67dfa39d2206dd6a3328
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [3]:
pip install transformers

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 48.2MB/s 
     |████████████████████████████████| 1.0MB 51.1MB/s 
     |████████████████████████████████| 870kB 47.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=c374939389d7e9538a8309bf1c83086914c3222275bcad07dd559590989d71ba
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [5]:

import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

Using TensorFlow backend.


**Loading the data and preprocessing the data**

In [0]:
data=pd.read_json("/content/drive/My Drive/LineUpX/Entity Recognition in Resumes.json",lines=True)

In [0]:
data.head()
data.drop(data.index[[81,167]],inplace=True)
#data.drop(data.index[167],inplace=True)

In [0]:
def split(message):
    return message.split()
    

In [0]:
data["resume"]=data["content"].apply(split)

In [0]:
def valuelabel(annotation,conten):
    try:
        label=["O"]*len(conten)
        a=len(annotation)
        for i in range(a):
            string=annotation[i]["points"][0]["text"]
            for ab in enumerate(conten):
                if ab[1] in string:
                    j=ab[0]
                    label[j]=annotation[i]["label"][0]
        return label
    except:
        print("hello")

In [0]:
data['labels'] = data.apply(lambda x: valuelabel(x['annotation'], x['resume']), axis=1)

In [12]:
data.shape

(218, 5)

In [0]:
tags=['O', 'Companies worked at', 'College Name', 'Designation', 'Skills', 'Degree', 'Name', 'Email Address', 'Graduation Year','Location','Years of Experience']

In [0]:
tags.append('X')
tags.append('[CLS]')
tags.append('[SEP]')

In [15]:
tag2idx = {t: i for i, t in enumerate(tags)}
tag2idx

{'College Name': 2,
 'Companies worked at': 1,
 'Degree': 5,
 'Designation': 3,
 'Email Address': 7,
 'Graduation Year': 8,
 'Location': 9,
 'Name': 6,
 'O': 0,
 'Skills': 4,
 'X': 11,
 'Years of Experience': 10,
 '[CLS]': 12,
 '[SEP]': 13}

In [0]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

1

In [0]:
vocabulary = "/content/drive/My Drive/LineUpX/bert-base-cased-vocab.txt"

In [0]:
tokenizer=BertTokenizer(vocab_file=vocabulary,do_lower_case=False)

In [0]:
max_len  = 2800


In [0]:
sentences=data["resume"].tolist()
labels=data["labels"].tolist()

**Remaining code its taken from Official GitHub page with hyperparameter optimization**

**Text Tokenizer**

In [22]:
tokenized_texts = []
word_piece_labels = []
i_inc = 0
for word_list,label in (zip(sentences,labels)):
    temp_lable = []
    temp_token = []
    
    # Add [CLS] at the front 
    temp_lable.append('[CLS]')
    temp_token.append('[CLS]')
    
    for word,lab in zip(word_list,label):
        token_list = tokenizer.tokenize(word)
        for m,token in enumerate(token_list):
            temp_token.append(token)
            if m==0:
                temp_lable.append(lab)
            else:
                temp_lable.append('X')  
                
    # Add [SEP] at the end
    temp_lable.append('[SEP]')
    temp_token.append('[SEP]')
    
    tokenized_texts.append(temp_token)
    word_piece_labels.append(temp_lable)
    
    if 5 > i_inc:
        print("No.%d,len:%d"%(i_inc,len(temp_token)))
        print("texts:%s"%(" ".join(temp_token)))
        print("No.%d,len:%d"%(i_inc,len(temp_lable)))
        print("lables:%s"%(" ".join(temp_lable)))
    i_inc +=1


No.0,len:425
texts:[CLS] A ##b ##his ##he ##k J ##ha Application Development Associate - A ##cc ##ent ##ure Bengal ##uru , Karnataka - Em ##ail me on Indeed : indeed . com / r / A ##b ##his ##he ##k - J ##ha / 10 ##e ##7 ##a ##8 ##c ##b ##7 ##32 ##b ##c ##43 ##a • To work for an organization which provides me the opportunity to improve my skills and knowledge for my individual and company ' s growth in best possible ways . Will ##ing to relocate to : Bangalore , Karnataka W ##OR ##K E ##X ##P ##ER ##IE ##NC ##E Application Development Associate A ##cc ##ent ##ure - November 2017 to Present Role : Currently working on Cha ##t - b ##ot . Dev ##elo ##ping Back ##end Oracle People ##S ##oft Que ##ries for the Bo ##t which will be triggered based on given input . Also , Training the b ##ot for different possible utter ##ance ##s ( Both positive and negative ) , which will be given as input by the user . E ##D ##UC ##AT ##ION B . E in Information science and engineering B . v . b college of 

**Set Token Embedding**

In [31]:
max_len=80
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids[0])

[  101   138  1830 27516  4638  1377   147  2328 22491  3273  9666   118
   138 19515  3452  3313  7756 12328   117 12247   118 18653 11922  1143
  1113 10364   131  5750   119  3254   120   187   120   138  1830 27516
  4638  1377   118   147  2328   120  1275  1162  1559  1161  1604  1665
  1830  1559 17101  1830  1665 25631  1161   794  1706  1250  1111  1126
  2369  1134  2790  1143  1103  3767  1106  4607  1139  4196  1105  3044
  1111  1139  2510  1105  1419   112   188  3213]


In [32]:
max_len=80
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags[50])

[12  6 11 11 11  6 11  3  3  3  0 10 11 10  3  0  0  5 11 11 11  3  0 11
  0 11  0  0  0 11  0  0  0  4  0  3  0 11 11  0  0  0  0  0  0 11 11  0
 11 11  0  7  0 11  3  4  0 11  7 11 11 11 11 11 11 11 11 11 11 11 11 11
  7 11 11 11 11 11 11 11]


In [0]:
attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
attention_masks[0];

In [0]:
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

***Test Train Split***

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

In [36]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(152, 66, 152, 66)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [0]:
batch_num = 16

In [0]:

# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

**MODEL TRAINING**

In [0]:
model_file_address = '/content/drive/My Drive/LineUpX/model/'

In [0]:
model = BertForTokenClassification.from_pretrained(model_file_address,num_labels=len(tag2idx))

In [0]:
model;

In [0]:
model.cuda();

In [0]:
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [0]:
epochs = 50
max_grad_norm = 1.0

In [0]:
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

**Fine Tuning**

In [0]:
FULL_FINETUNING = True

In [0]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [0]:
model.train();

In [51]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 152
  Batch size = 16
  Num steps = 500



Epoch:   2%|▏         | 1/50 [00:01<01:09,  1.41s/it]

Train loss: 0.20402388440238106



Epoch:   4%|▍         | 2/50 [00:02<01:07,  1.41s/it]

...

Train loss: 0.007228574508594142



Epoch:  98%|█████████▊| 49/50 [01:08<00:01,  1.39s/it]

Train loss: 0.006821632876785265



Epoch: 100%|██████████| 50/50 [01:09<00:00,  1.40s/it]

Train loss: 0.005269428792720039


Model Evalution

In [0]:
model.eval();

In [53]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
bert_out_address="/content/drive/My Drive/LineUpX"
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =66
  Batch size = 16
f1 socre: 0.592043
Accuracy score: 0.797997
***** Eval results *****

                     precision    recall  f1-score   support

             Skills     0.2556    0.3091    0.2798       110
       College Name     0.2222    0.1143    0.1509        35
        Designation     0.4590    0.4275    0.4427       131
               Name     0.8165    0.8396    0.8279       106
Years of Experience     0.0909    0.1250    0.1053         8
             Degree     0.3793    0.2683    0.3143        41
      Email Address     0.8788    0.9022    0.8904       225
Companies worked at     0.4918    0.5000    0.4959        60
           Location     0.7857    0.5500    0.6471        20
    Graduation Year     0.0000    0.0000    0.0000         2

          micro avg     0.5893    0.5949    0.5920       738
          macro avg     0.5987    0.5949    0.5943       738

f1 socre: 0.592043
Accuracy score: 0.797997
